In [1]:
import sys 
import datetime
import h5py
sys.path.insert(0, '../')

from constants import * 
import datasets
import loss_fns
import models
import hickle
import util

Using TensorFlow backend.


In [2]:
train_parser = util.get_train_parser()
args = train_parser.parse_args(['--epochs', str(1),
                                '--model_name', 'fcn_crnn',
                                '--dataset', 'full', 
                                '--num_classes', str(4),
                                '--country', 'ghana',
                                '--hdf5_filepath', '/home/lijing/croptype_data_local/data/ghana/data.hdf5',
                                '--batch_size', str(1),
                                '--hidden_dims', str(4),
                                '--crnn_num_layers', str(1),
                                '--use_s1', str(True),
                                '--use_s2', str(True),
                                '--sample_w_clouds', str(False),
                                '--include_clouds', str(True),
                                '--include_doy', str(True),
                                '--bidirectional', str(False), 
                                '--shuffle', str(False),
                                '--normalize', str(False),
                                '--apply_transforms', str(False),
                                '--least_cloudy', str(False),
                                '--s2_num_bands', str(4)])
    
# load in data generator
dataloaders = datasets.get_dataloaders(args.grid_dir, args.country, args.dataset, args)
model_name =  'random_forest'
save_path =  '/home/lijing/croptype_data_local/data/ghana/pixel_array.h5'

In [11]:
def hdf5_pixelwise(model_name, save_path, args=None, dataloaders = None):
    # transfer data.hdf5 to pixel_array.h5
    if model_name in ['logreg', 'random_forest']:
        pixel_data = {}
        for split in ['train','test','val']:
            print(split)
            dl = dataloaders[split]
            Input = None
            Target = None
            Cloudmask = None
            for inputs, targets, cloudmasks in dl:
                if Input is None:
                    Input = np.copy(inputs)
                    Target = np.copy(targets)
                    Cloudmask = np.copy(cloudmasks)
                else:
                    Input = np.vstack((Input, inputs))
                    Target = np.vstack((Target, targets))
                    Cloudmask = np.vstack((Cloudmask, cloudmasks))
                    
                if Input.shape[0]%100 == 0:
                    print(Input.shape[0])
            # resize
            Target = Target.reshape(Target.shape[0],Target.shape[1],-1)
            Target = np.swapaxes(Target, 1, 2)
            Target = Target.reshape(-1,Target.shape[2])

            Input = Input.reshape(Input.shape[0],Input.shape[1],Input.shape[2],-1)
            Input = np.swapaxes(Input, 0, 2)
            Input = Input.reshape(Input.shape[0],Input.shape[1],-1)
            Input = np.swapaxes(Input, 0, 1)
            # bytime: rgbrgbrgb
            Input = Input.reshape(-1,Input.shape[2]).T

            Cloudmask = Cloudmask.reshape(-1,Cloudmask.shape[4])
            
            pixel_data[split] = [Input,Target,Cloudmask]
            
        hickle.dump(pixel_data, save_path)

In [ ]:
hdf5_pixelwise(model_name, save_path, args, dataloaders)

train
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
